### ROCKET
RandOm Convolutional KErnel Transform functions for univariate and multivariate time series.

#### 1. Load the data

In [1]:
RAW_DATA_DIR = '../data/raw/'
INSTANCES_DIR = '../data/instances/'
INFERENCE_DIR = '../data/inference/'
STATES = ['normal', 
          'backlash1', 'backlash2',
          'lackLubrication1', 'lackLubrication2',
          'spalling1', 'spalling2', 'spalling3', 'spalling4',
          'spalling5', 'spalling6', 'spalling7', 'spalling8']
LOADS= ['20kg', '40kg', '-40kg']
SMOOTHING_PE_WINDOW_SIZE = 20
SMOOTHING_CUR_WINDOW_SIZE = 15

In [2]:
import sys
sys.path.append('../')
from rocket_rag.utils import *

import random

In [3]:
load = '20kg'
load_state_dict = {load: [os.listdir(os.path.join(INSTANCES_DIR, load, state)) for state in STATES] for load in LOADS}

random.seed(42)
rand_state = random.sample(STATES, 1)[0]
rand_idx = random.randint(0, 10)
random_sample = load_state_dict[load][STATES.index(rand_state)][rand_idx]

In [4]:
random_sample

'spalling6_20_10_2.csv'

In [5]:
ts_filename = os.path.join(INSTANCES_DIR, load, rand_state, random_sample)
cur_ts = fit(ts_filename, 
             field='current', 
             smooth=True, 
             smooth_ws=SMOOTHING_CUR_WINDOW_SIZE, 
             tolist=True, 
             verbo=True)

2024-03-15 04:17:16.102 | INFO     | rocket_rag.utils:fit:123 - time series extracted SUCCESSFULLY.


#### 2. ROCKET implementation

In [6]:
import torch
import torch.nn as nn

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:
class ROCKET(nn.Module):
    """
    """
    
    def __init__(self, c_in, seq_len, n_kernels=10_000, kss=[7, 9, 11], device=None, verbose=False):
        super().__init__()
        kss = [ks for ks in kss if ks < seq_len]
        convs = nn.ModuleList()
        for i in range(n_kernels):
            ks = np.random.choice(kss)
            dilation = 2**np.random.uniform(0, np.log2((seq_len - 1) // (ks - 1)))
            padding = int((ks - 1) * dilation // 2) if np.random.randint(2) == 1 else 0
            weight = torch.randn(1, c_in, ks)
            weight -= weight.mean()
            bias = 2 * (torch.rand(1) - .5)
            layer = nn.Conv1d(c_in, 1, ks, padding=2 * padding, dilation=int(dilation), bias=True)
            layer.weight = torch.nn.Parameter(weight, requires_grad=False)
            layer.bias = torch.nn.Parameter(bias, requires_grad=False)
            convs.append(layer)
        self.convs = convs
        self.n_kernels = n_kernels
        self.kss = kss
        self.to(device=device)
        self.verbose=verbose
    
    def forward(self, x):
        _output = []
        for i in range(self.n_kernels):
            out = self.convs[i](x).cpu()
            _max = out.max(dim=-1)[0]
            _ppv = torch.gt(out, 0).sum(dim=-1).float() / out.shape[-1]
            _output.append(_max)
            _output.append(_ppv)
        return torch.cat(_output, dim=-1)

In [9]:
rocket = ROCKET(c_in=1, seq_len=len(cur_ts), device=device)

In [10]:
cur_ts_tensor = torch.tensor([cur_ts], dtype=torch.float32)
cur_ts_tensor.unsqueeze(1)

tensor([[[0.5581, 0.4940, 0.4547, 0.5040, 0.5723, 0.6208, 0.6423, 0.6525,
          0.6490, 0.6350, 0.6273, 0.6236, 0.6150, 0.6080, 0.6017, 0.5999,
          0.5935, 0.5929, 0.6051, 0.6016, 0.5819, 0.5626, 0.5506, 0.5486,
          0.5501, 0.5553, 0.5604, 0.5685, 0.5792, 0.5903, 0.5978, 0.6070,
          0.6240, 0.6450, 0.6563, 0.6659, 0.6796, 0.6960, 0.7125, 0.7200,
          0.7259, 0.7352, 0.7433, 0.7454, 0.7506, 0.7503, 0.7499, 0.7494,
          0.7456, 0.7433, 0.7388, 0.7321, 0.7288, 0.7222, 0.7152, 0.7056,
          0.6990, 0.6949, 0.6914, 0.6901, 0.6852, 0.6855, 0.6915, 0.6940,
          0.7034, 0.7062, 0.7104, 0.7145, 0.7162, 0.7177, 0.7149, 0.7194,
          0.7194, 0.7231, 0.7228, 0.7278, 0.7295, 0.7320, 0.7276, 0.7244,
          0.7161, 0.7108, 0.7039, 0.7030, 0.6988, 0.6993, 0.6958, 0.6934,
          0.6921, 0.6936, 0.6938, 0.6861, 0.6835, 0.6806, 0.6815, 0.6843,
          0.6863, 0.6852, 0.6858, 0.6855, 0.6884, 0.6835, 0.6848, 0.6833,
          0.6913, 0.6916, 0.6929, 0.69

In [79]:
rocket_trans = rocket(cur_ts_tensor)
print(rocket_trans)

tensor([-0.6733,  0.0000,  0.8113,  ...,  0.0652,  1.3030,  1.0000])


In [81]:
rocket_trans.shape

torch.Size([20000])